### <font color="darkpurple">If this kernel helps you,please upvote it😁😁😁

## <font color="Red"> Contents
*    <font color="Green"> Loading Dataset
*    <font color="Green"> Importing libraries
*     <font color="Green"> Removing the unnecessary columns
*     <font color="Green"> checking the number of null values in each column
        *  <font color="Green">  Notice--Missing Values
        *   <font color="Green">  Adjusting the rate column (/5) type     
        *   <font color="Green"> Adjusting dish_liked column by most frequent by SimpleImputer
        *   <font color="Green">  Duplicated rows
        *   <font color="Green"> Dropping Duplicated rows
* <font color="darkpink"> Finding max branches' restaurant
* <font color="darkpink">  Most voted Restaurant
* <font color="darkpink">  Most rated Restaurant
* <font color="Green"> EDA
* <font color="Green"> Data Visulisation
* <font color="Green">  Creating Model
* <font color="Green"> DecisionTreeRegressor
* <font color="Green"> RandomForestRegressor

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score

In [ ]:
data=pd.read_csv("/kaggle/input/zomato-bangalore-restaurants/zomato.csv")
data.head(3)

## <font color="orange"> Removing the unnecessary columns
    * We are removing the url,address and phone as they are not contributing to pprox_cost


In [ ]:
print(len([data.menu_item[i]=='[]' for i in range(len(data))]),len(data))

# since all rows are empty,we are going to remove the columns
data.drop("menu_item",axis=1,inplace=True)

data.drop(["url","address","phone"],axis=1,inplace=True)

## <font color="orange"> checking the number of null values in each column

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

## <font color="orange"> Notice--Missing Values

* #### <font color="green"> since null values in location and cuisines are less in number 
*     <font color="green">we cant give any most frequent value to location and cuisines to missing values as it changes its popularity
*     <font color="green"> we cannot remove all rows of missing values in dish_liked as it is larger in count.so we replace with most
       frequent item
*     <font color="green">  for approx_cost,lets take mean value of this column to replace the missing values
*     <font color="green">   for rest_type,we remove those values as it is smaller count   
*     <font color="green">  for rate,we gonna replace with mean value   

In [ ]:
# we are removing the null values' rows in any of the below columns
data.dropna(subset=["location","cuisines","rest_type"],how="any",inplace=True)

### <font color="orange"> Adjusting the rate column  (/5) type

In [ ]:
data.rate=data.rate.astype(str)  

data.rate=data.rate.loc[data.rate!="NEW"]  # we have to select the integer string values only

data.rate=data.rate.loc[data.rate!="-"]

data.rate=data.rate.str.replace("/5","")  # removing "/5" in integer string

data.rate=data.rate.apply(lambda x:float(x)) # converting to float

myimputer=SimpleImputer(strategy="mean") # replacing null values with mean

data.rate=pd.DataFrame(myimputer.fit_transform(np.array(data.rate).reshape(-1,1)))

### <font color="orange"> Adjusting dish_liked column by most frequent

In [ ]:
# "dish_liked" column
# Now we are going to use simple imputer to replace the missing values

myimputer=SimpleImputer(strategy="most_frequent")

data.dish_liked=pd.DataFrame(myimputer.fit_transform(np.array(data.dish_liked).reshape(-1,1)))

data.head(2)

### <font color="orange"> Adjusting approx_cost column

In [ ]:
# for approx_cost column

# we have convert it to float type


data['approx_cost(for two people)']=data['approx_cost(for two people)'].str.replace(",","") # removing , in integers

data['approx_cost(for two people)']=data['approx_cost(for two people)'].apply(lambda x:float(x)) # converting to float

myimputer=SimpleImputer(strategy="mean")  # replacing the missing value with mean value

data['approx_cost(for two people)']=pd.DataFrame(myimputer.fit_transform(np.array(data['approx_cost(for two people)']).reshape(-1,1)))

In [ ]:
data.dropna(how="any",subset=list(data.columns),inplace=True)

### <font color="orange">Duplicated rows

In [ ]:
data[data.duplicated()]
# since we have single duplicated row,we neglect it or we can remove the row

### <font color="orange">Dropping Duplicated rows

In [ ]:
data.drop_duplicates(keep="first").head(2)  # we keep only the first occured row

### <font color="orange"> Finding max branch restaurant

In [ ]:
data2=data.groupby("name").name.agg(["count"]).reset_index()

# renaming columns as count column which is obtained is function name.so we cant use it

data2.rename(columns={"name":"name","count":"repetition"},inplace=True)

print(data2.head())

data2.name[data2.repetition.max()] # the restaurant having more number of branches

### <font color="orange"> Most voted Restaurant

In [ ]:
maximum=data.votes.max()

# renaming columns as count column which is obtained is function name.so we cant use it

data.name[data.votes==maximum]

# The Byg Brewski Brewing Company restaurant has highest votes with 3 different branches

### <font color="orange"> Most rated Restaurant

In [ ]:
maximum=data.rate.max()

print(maximum)

# renaming columns as count column which is obtained is function name.so we cant use it

len(data.name[data.rate==maximum])

# There 55 highest rated restaurants

## <font color="red">EDA
    * lets find the famous restaurant from various features(ratings,votes)

### <font color="Red"> Data Visualisation

In [ ]:
plt.figure(1,(20,10))
sns.countplot(data.book_table,data=data,hue="listed_in(type)")
plt.show()

# most of restaurants dont allow booking table 

In [ ]:
plt.figure(1,(20,10))
sns.countplot(data.online_order,data=data,hue="listed_in(type)")
plt.show()

### <font color="orange">Notice from plot
*    <font color="green"> Most of the online orders have delivery
*    <font color="green"> Most of the Dine-Out doesnt deliver the food
*    <font color="green"> Desserts are not delivered in most restaurants
    
    

In [ ]:
plt.figure(1,(30,20))
ax=sns.countplot(data["listed_in(city)"],data=data)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90,ha="right")
plt.show()

### <font color="orange">Notice from plot
*    <font color="green"> BTM has highest number of restaurants
*    <font color="green"> Koramangala 4th,5th,6th blocks are next to it
*    <font color="green"> Banashankari,New Bel Road has low number of restaurants

In [ ]:
#Restaurant Type
ax=sns.countplot(data['rest_type'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="right")
fig = plt.gcf()   # it is used to get the current figure.if we dont get,figure() function creates the one
fig.set_size_inches(15,15)   # sets the inches of rest_type
plt.title('Restuarant Type')

##### <font color="green"> THe casual dining and cafe are in more number of restaurants

In [ ]:
plt.figure(1,(10,10))
data2=data.name.value_counts()[:15]
ax=sns.barplot(x=data2.index,y=data2,palette="bright") # different types of palettes--bright,muted,dark,colorblind
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.ylabel("rate")
plt.show()

In [ ]:
plt.figure(1,(15,7))
ax=sns.countplot(data.rate,data=data,hue="listed_in(type)")
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

### <font color="red"> Creating Model

### <font color="orange"> Encoding the data

In [ ]:
#Encode the input Variables
def Encode(dat):
    for column in dat.columns[~dat.columns.isin(['rate', 'cost', 'votes'])]:
        dat[column] = dat[column].factorize()[0]   # it used for the numeric representation of an array to identify the distinct values
    return dat

In [ ]:
finaldata = Encode(data.copy())
finaldata.head()

### <font color="orange"> Correlation

In [ ]:
corr=finaldata.corr(method="pearson")
plt.figure(1,(15,8))
sns.heatmap(corr,annot=True)
plt.show()

In [ ]:

x = finaldata.iloc[:,[2,3,8,9,11]]
y = finaldata['approx_cost(for two people)']
#Getting Test and Training Set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=101)


### <font color="orange"> DecisionTree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor(min_samples_leaf=0.01,random_state=101) 
# when min_smaples_leaf is int,then it is number required to considered as leaf
# if it is float, required number of leaf =len(data)*0.00007  her we get 3
model.fit(x_train,y_train)
pred=model.predict(x_test)
r2_score(pred,y_test)


## <font color="orange"> RndomforestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=500,random_state=101,min_samples_leaf=.0001)
rf.fit(x_train,y_train)   # n_estimators--The number of trees in forest
y_predict=rf.predict(x_test)
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)